In [13]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,LeakyReLU
from keras.regularizers import l2

# from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from keras import mixed_precision
from keras.optimizers import Adam, RMSprop, SGD, Adagrad, Adadelta, Nadam


from keras.applications import InceptionV3


In [14]:

# read the CSV file
df_full = pd.read_csv('/mnt/data/MS39_Raw/CSO2/metadata/new_skanda_230320231356.csv', low_memory=False)

# Filter by unique Image ID 
df_byImageID= df_full.drop_duplicates(subset=['ImageID'])
# Filter the DataFrame based on the 'Resolution' column and select the first 1500 rows
df_byImageID= df_byImageID[df_byImageID['Resolution'].str.contains("1024x1800")].head(200)

In [15]:
# Load images from file paths
def load_images(df):
    images = []
    for file_path in df['FilePath']:
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)  # Assuming grayscale images
        img = cv2.resize(img, (128, 225))  # Resize the image (change dimensions as needed)
        img = np.stack((img,)*3, axis=-1)  # Stack the grayscale image to create a 3-channel image
        img = np.transpose(img, (1, 0, 2))  # Transpose the image to swap height and width
        images.append(img)
    return np.array(images)

# Load images into memory
images = load_images(df_byImageID)

# Separate features (images) and target (K2) values
X = images
y = df_byImageID['K2'].values

# Filter X and y simultaneously, removing rows with NaNs in y
X_filtered, y_filtered = zip(*[(x, target) for x, target in zip(X, y) if not np.isnan(target)])

# Convert the filtered X and y back to numpy arrays
X_filtered = np.array(X_filtered)
y_filtered = np.array(y_filtered)

# Split the dataset into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(X_filtered, y_filtered, test_size=0.3, random_state=42)

# Normalize the image data
train_images = train_images / 255.0
test_images = test_images / 255.0

#3.5

# Create a mask for non-NaN values in train_labels
non_nan_mask = ~np.isnan(train_labels)

# Filter train_images and train_labels using the mask
train_images_filtered = train_images[non_nan_mask]
train_labels_filtered = train_labels[non_nan_mask]

# Check if there are any NaN values in train_labels_filtered
print("Are there NaN values in train_labels_filtered?", np.isnan(train_labels_filtered).any())

Are there NaN values in train_labels_filtered? False


In [16]:
from keras.optimizers import Adam, RMSprop, SGD, Adagrad, Adadelta, Nadam
from keras.applications import InceptionV3
from keras.layers import Dense, Dropout, Flatten, LeakyReLU
from keras.models import Sequential
from keras.regularizers import l2
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
import numpy as np
from sklearn.metrics import make_scorer

from keras.models import clone_model

def create_model(optimizer='Adam', lr=0.001):
    inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 225, 3))

    for layer in inception.layers[:-10]:
        layer.trainable = False

    for layer in inception.layers[-10:]:
        layer.trainable = True

    model = Sequential([
        inception,
        Flatten(),
        Dense(256, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
        LeakyReLU(alpha=0.1),
        Dropout(0.5),
        Dense(128, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)),
        LeakyReLU(alpha=0.1),
        Dropout(0.5),
        Dense(1, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))
    ])

    if optimizer == 'Adam':
        opt = Adam(lr=lr)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=lr)
    else:
        raise ValueError("Invalid optimizer name")

    model_clone = clone_model(model)
    model_clone.set_weights(model.get_weights())
    model_clone.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])
    
    return model_clone

keras_model = KerasRegressor(build_fn=create_model, verbose=1)

param_dist = {
    'optimizer': ['Adam', 'Nadam'],
    'lr': [0.001, 0.0001],
    'epochs': [300, 500, 700],
}

random_search = RandomizedSearchCV(estimator=keras_model, param_distributions=param_dist, scoring=wrapped_scorer, n_iter=6, cv=3, random_state=42)
random_search_result = random_search.fit(train_images_filtered, train_labels_filtered)

print("Best: %f using %s" % (random_search_result.best_score_, random_search_result.best_params_))


/tmp/ipykernel_1100/4185192983.py:47: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_model = KerasRegressor(build_fn=create_model, verbose=1)
/mnt/data/skanda/mambaforge/envs/gpu/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/700
3/3 [==============================] - 4s 21ms/step - loss: 1828.2507 - mae: 41.4438
Epoch 2/700
3/3 [==============================] - 0s 22ms/step - loss: 435.8944 - mae: 18.1029
Epoch 3/700
3/3 [==============================] - 0s 22ms/step - loss: 377.5416 - mae: 15.5246
Epoch 4/700
3/3 [==============================] - 0s 22ms/step - loss: 385.6389 - mae: 15.5502
Epoch 5/700
3/3 [==============================] - 0s 22ms/step - loss: 381.4433 - mae: 15.8001
Epoch 6/700
3/3 [==============================] - 0s 21ms/step - loss: 211.9180 - mae: 11.7148
Epoch 7/700
3/3 [==============================] - 0s 22ms/step - loss: 243.6521 - mae: 12.1031
Epoch 8/700
3/3 [==============================] - 0s 22ms/step - loss: 244.0390 - mae: 12.5050
Epoch 9/700
3/3 [==============================] - 0s 21ms/step - loss: 252.6620 - mae: 12.8775
Epoch 10/700
3/3 [==============================] - 0s 21ms/step - loss: 217.0549 - mae: 11.5691
Epoch 11/700
3/3 [====================

/mnt/data/skanda/mambaforge/envs/gpu/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/nadam.py:89: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/700
3/3 [==============================] - 4s 25ms/step - loss: 2611.3584 - mae: 47.8167
Epoch 2/700
3/3 [==============================] - 0s 29ms/step - loss: 366.6924 - mae: 16.1972
Epoch 3/700
3/3 [==============================] - 0s 24ms/step - loss: 271.7895 - mae: 12.8698
Epoch 4/700
3/3 [==============================] - 0s 24ms/step - loss: 326.0656 - mae: 13.9506
Epoch 5/700
3/3 [==============================] - 0s 22ms/step - loss: 313.9797 - mae: 14.6517
Epoch 6/700
3/3 [==============================] - 0s 23ms/step - loss: 283.5354 - mae: 13.1567
Epoch 7/700
3/3 [==============================] - 0s 22ms/step - loss: 276.0516 - mae: 12.5909
Epoch 8/700
3/3 [==============================] - 0s 23ms/step - loss: 285.0808 - mae: 13.3274
Epoch 9/700
3/3 [==============================] - 0s 24ms/step - loss: 240.1725 - mae: 11.9153
Epoch 10/700
3/3 [==============================] - 0s 23ms/step - loss: 257.9138 - mae: 13.2598
Epoch 11/700
3/3 [====================

In [17]:
random_search.best_estimator_
